In [0]:
import json
from pandas import json_normalize
import pandas as pd
import numpy as np

In [0]:
def formatting_squad(dataset_file, display=False):
    with open(dataset_file, encoding='utf-8') as f:
        # print(json.dumps(f))
        raw_data = json.load(f)
        raw_data = raw_data['data']
        raw_data = json_normalize(raw_data)['paragraphs']
        context = []
        question = []
        answer_start = []
        text = []
        for i in range(len(raw_data)): # paragraphs
            for j in range(len(raw_data[i])): #qas
                for k in range(len(raw_data[i][j]['qas'])):
                    if (len(raw_data[i][j]['qas'][k]['answers']) != 0):
                        question.append(raw_data[i][j]['qas'][k]['question'])
                        answer_start.append(raw_data[i][j]['qas'][k]['answers'][0]['answer_start'])
                        text.append(raw_data[i][j]['qas'][k]['answers'][0]['text'])
                        context.append(raw_data[i][j]['context'])
                    else:
                        continue
        data = pd.DataFrame({"context":context, "question": question, "answer_start": answer_start, "text": text})
#         print(data.head())
        if display is True:
            print(data.shape)
        return data

In [0]:
train_data = formatting_squad(r"train.json")
train_data = train_data[train_data['context'].apply(lambda x: len(x)<500)].reset_index(drop=True)
# print(train_data.shape)
# train_data.head(3)

In [0]:
valid_data = formatting_squad(r"valid.json")
# valid_data = valid_data[valid_data['context'].apply(lambda x: len(x)<500)].reset_index(drop=True)
valid_data = valid_data.reset_index(drop=True)
# print(valid_data.shape)
# valid_data.head(3)

--------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------

In [5]:
!pip install transformers

In [0]:
import transformers
from torch import nn
import torch
from torch.utils.data import Dataset, DataLoader
from transformers.optimization import AdamW
from tqdm import trange, tqdm_notebook
# import numpy as np

In [0]:
from transformers.modeling_camembert import CamembertModel
from transformers.tokenization_camembert import CamembertTokenizer

In [0]:
# import math
# # import torch.nn as nn

# class PositionalEncoding(nn.Module):
#     def __init__(self, d_model, dropout=0.1, max_len=5000):
#         super(PositionalEncoding, self).__init__()
#         self.dropout = nn.Dropout(p=dropout)
#         self.d_model = d_model

#         pe = torch.zeros(max_len, d_model)
#         position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
#         div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
#         pe[:, 0::2] = torch.sin(position * div_term)
#         pe[:, 1::2] = torch.cos(position * div_term)
#         pe = pe.unsqueeze(0).transpose(0, 1)
#         self.register_buffer('pe', pe)

#     def forward(self, x, seq_len = 768, mask = None):
#         pos_emb = self.pe[:, :seq_len]
#         x = x * mask[:, :, None].float()
#         x = x + pos_emb
#         return x

In [0]:
# class CamemBERTQA(nn.Module):
#     def __init__(self,bert_type, hidden_size, num_labels, num_inter_layers=1, heads = 12, do_lower_case = True):
#         super(CamemBERTQA, self).__init__()
#         self.do_lower_case = do_lower_case
#         self.bert_type = bert_type
#         self.hidden_size = hidden_size
#         self.num_labels = num_labels       
#         self.num_inter_layers = num_inter_layers
#         self.camembert = CamembertModel.from_pretrained(self.bert_type)

#         # ---------------------------------------------------------------------------------------------------------------------
#         self.d_model = self.hidden_size # 768
#         dim_feedforward = self.hidden_size * 4  # 2048
#         dropout = 0.1
#         self.pos_emb = PositionalEncoding(d_model = self.d_model, dropout = dropout)
#         self.transformer_inter = nn.ModuleList(
#             [nn.TransformerEncoderLayer(d_model = self.d_model, nhead = heads, dim_feedforward = 2048, dropout = dropout)  #dim_feedforward=2048, dropout=0.1)
#              for _ in range(num_inter_layers)])
#         # ---------------------------------------------------------------------------------------------------------------------
#         self.qa_outputs = nn.Linear(self.hidden_size, self.num_labels)

#     def forward(self, input_ids, mask=None):
#         bert_output = self.camembert(input_ids = input_ids) # input_ids is a tensor
        
#         # ---------------------------------------------------------------------------------------------------------------------
#         seq_len = self.hidden_size
#         x = self.pos_emb(x = bert_output, seq_len = seq_len, mask = None)

#         for i in range(self.num_inter_layers):
#             x = self.transformer_inter[i](i, x, x, 1 - mask)  # all_tokens * max_tokens * dim
#         output = self.layer_norm(x) # Transformer also normalizes the outputs from each layer.
#         # ---------------------------------------------------------------------------------------------------------------------

#         sequence_output = output[0]   #(None, seq_len, hidden_size)
#         logits = self.qa_outputs(sequence_output) #(None, seq_len, hidden_size)*(hidden_size, 2)=(None, seq_len, 2)
#         start_logits, end_logits = logits.split(1, dim=-1)    #(None, seq_len, 1), (None, seq_len, 1)
#         start_logits = start_logits.squeeze(-1)  #(None, seq_len)
#         end_logits = end_logits.squeeze(-1)    #(None, seq_len)
#         outputs = (start_logits, end_logits,) 
#         return x

In [0]:
# import logging
# logger = logging.getLogger("tensor_shapes")
# handler = logging.StreamHandler()
# formatter = logging.Formatter(
#         '%(message)s')
# handler.setFormatter(formatter)
# logger.addHandler(handler)
# # if you want the model to continuously print tensor shapes, set to DEBUG!
# logger.setLevel(1)

In [0]:
# import inspect
# def getclass():
#     stack = inspect.stack()
#     return stack[3][0].f_locals["self"].__class__

# # A helper function to check how tensor sizes change
# def log_size(tsr: torch.Tensor, name: str):
#     cls = getclass()
#     logger.log(level=cls.level, msg=f"[{cls.__name__}] {name} size={tsr.shape}")

In [0]:
from enum import IntEnum
# # Control how much debugging output we want
# class TensorLoggingLevels(IntEnum):
#     attention = 1
#     attention_head = 2
#     multihead_attention_block = 3
#     enc_dec_block = 4
#     enc_dec = 5
class Dim(IntEnum):
    batch = 0
    seq = 1
    feature = 2

In [0]:
import math

class ScaledDotProductAttention(nn.Module):
    # level = TensorLoggingLevels.attention # Logging level: 
    def __init__(self, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)

    def forward(self, q, k, v, mask=None):
        d_k = k.size(-1) # get the size of the key
        assert q.size(-1) == d_k

        # compute the dot product between queries and keys for
        # each batch and position in the sequence
        attn = torch.bmm(q, k.transpose(Dim.seq, Dim.feature)) # (Batch, Seq, Seq)
        # we get an attention score between each position in the sequence
        # for each batch

        # scale the dot products by the dimensionality (see the paper for why we do this!)
        attn = attn / math.sqrt(d_k)
        # normalize the weights across the sequence dimension
        # (Note that since we transposed, the sequence and feature dimensions are switched)
        attn = torch.exp(attn)
        # log_size(attn, "attention weight") # (Batch, Seq, Seq)
        
        # fill attention weights with 0s where padded
        if mask is not None: attn = attn.masked_fill(mask, 0)
        attn = attn / attn.sum(dim=-1, keepdim=True)
        attn = self.dropout(attn)
        output = torch.bmm(attn, v) # (Batch, Seq, Feature)
        # log_size(output, "attention output size") # (Batch, Seq, Seq)
        return output

In [0]:
class AttentionHead(nn.Module):
    # level = TensorLoggingLevels.attention_head
    def __init__(self, d_model, d_feature, dropout=0.1):
        super().__init__()
        # We will assume the queries, keys, and values all have the same feature size
        self.attn = ScaledDotProductAttention(dropout)
        self.query_tfm = nn.Linear(d_model, d_feature)
        self.key_tfm = nn.Linear(d_model, d_feature)
        self.value_tfm = nn.Linear(d_model, d_feature)

    def forward(self, queries, keys, values, mask=None):
        Q = self.query_tfm(queries) # (Batch, Seq, Feature)
        K = self.key_tfm(keys) # (Batch, Seq, Feature)
        V = self.value_tfm(values) # (Batch, Seq, Feature)
        # log_size(Q, "queries, keys, vals")
        # compute multiple attention weighted sums
        x = self.attn(Q, K, V)
        return x

In [0]:
class MultiHeadAttention(nn.Module):
    # level = TensorLoggingLevels.multihead_attention_block
    def __init__(self, d_model, d_feature, n_heads, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.d_feature = d_feature
        self.n_heads = n_heads
        # in practice, d_model == d_feature * n_heads
        assert d_model == d_feature * n_heads

        # Note that this is very inefficient:
        # I am merely implementing the heads separately because it is 
        # easier to understand this way
        self.attn_heads = nn.ModuleList([
            AttentionHead(d_model, d_feature, dropout) for _ in range(n_heads)
        ])
        self.projection = nn.Linear(d_feature * n_heads, d_model) 
    
    def forward(self, queries, keys, values, mask=None):
        # log_size(queries, "Input queries")
        x = [attn(queries, keys, values, mask=mask) # (Batch, Seq, Feature)
             for i, attn in enumerate(self.attn_heads)]
        # log_size(x[0], "output of single head")
        
        # reconcatenate
        x = torch.cat(x, dim=Dim.feature) # (Batch, Seq, D_Feature * n_heads)
        # log_size(x, "concatenated output")
        x = self.projection(x) # (Batch, Seq, D_Model)
        # log_size(x, "projected output")
        return x

In [0]:
class LayerNorm(nn.Module):
    def __init__(self, d_model, eps=1e-8):
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(d_model))
        self.beta = nn.Parameter(torch.zeros(d_model))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.gamma * (x - mean) / (std + self.eps) + self.beta

In [0]:
class EncoderBlock(nn.Module):
    # level = TensorLoggingLevels.enc_dec_block
    # def __init__(self, d_model=512, d_feature=64,
                #  d_ff=2048, n_heads=8, dropout=0.1):
    def __init__(self, d_model=768, d_feature=64,
                 d_ff=3072, n_heads=12, dropout=0.1):
        super().__init__()
        self.attn_head = MultiHeadAttention(d_model, d_feature, n_heads, dropout)
        self.layer_norm1 = LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        self.position_wise_feed_forward = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model),
        )
        self.layer_norm2 = LayerNorm(d_model)
        
    def forward(self, x, mask=None):
        # log_size(x, "Encoder block input")
        att = self.attn_head(x, x, x, mask=mask)
        # log_size(x, "Attention output")
        # Apply normalization and residual connection
        x = x + self.dropout(self.layer_norm1(att))
        # Apply position-wise feedforward network
        pos = self.position_wise_feed_forward(x)
        # log_size(x, "Feedforward output")
        # Apply normalization and residual connection
        x = x + self.dropout(self.layer_norm2(pos))
        # log_size(x, "Encoder size output")
        return x

In [0]:
class TransformerEncoder(nn.Module):
    # level = TensorLoggingLevels.enc_dec
    # def __init__(self, n_blocks=6, d_model=512,
                #  n_heads=8, d_ff=2048, dropout=0.1):
    def __init__(self, n_blocks=2, d_model=768 # 768
                 , n_heads=12, d_ff=3072, dropout=0.1):
        super().__init__()
        self.encoders = nn.ModuleList([
            EncoderBlock(d_model=d_model, d_feature=d_model // n_heads,
                         d_ff=d_ff, dropout=dropout)
            for _ in range(n_blocks)
        ])
    
    def forward(self, x: torch.FloatTensor, mask=None):
        for encoder in self.encoders:
            x = encoder(x)
        return x

In [0]:
class PositionalEmbedding(nn.Module):
    # level = 1
    def __init__(self, d_model, max_len=512):
        super().__init__()        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.weight = nn.Parameter(pe, requires_grad=False)
        
    def forward(self, x):
        return self.weight[:, :x.size(1), :] # (1, Seq, Feature)

In [0]:
class CamemBERTQA(nn.Module):
    def __init__(self,bert_type, hidden_size, num_labels, n_blocks = 2, n_heads = 12, dropout=0.1):
        super(CamemBERTQA, self).__init__()
        self.bert_type = bert_type
        self.hidden_size = hidden_size
        self.num_labels = num_labels
        self.n_blocks = n_blocks
        self.n_heads = n_heads
        self.dropout = dropout
        self.camembert = CamembertModel.from_pretrained(self.bert_type)
        self.transformer = TransformerEncoder(n_blocks = self.n_blocks, d_model = self.hidden_size
                                              , n_heads=self.n_heads, d_ff=3072, dropout=self.dropout)
        self.qa_outputs = nn.Linear(self.hidden_size, self.num_labels)

    def forward(self, input_ids, mask=None):
        output = self.camembert(input_ids = input_ids) # input_ids is a tensor
        sequence_output = self.transformer(output[0]) # Transformer
        logits = self.qa_outputs(sequence_output) #(None, seq_len, hidden_size)*(hidden_size, 2)=(None, seq_len, 2)
        start_logits, end_logits = logits.split(1, dim=-1)    #(None, seq_len, 1), (None, seq_len, 1)
        start_logits = start_logits.squeeze(-1)  #(None, seq_len)
        end_logits = end_logits.squeeze(-1)    #(None, seq_len)
        outputs = (start_logits, end_logits,) 
        return outputs

### Loss function

In [0]:
def loss_func(out, s_target, e_target):
    criterion = nn.CrossEntropyLoss()
    s_loss = criterion(out[0], s_target)
    e_loss = criterion(out[1], e_target)
    total_loss = s_loss+e_loss
    return total_loss

# Dataloader for train and eval set:


In [0]:
class BertDatasetModule(Dataset):
    def __init__(self, tokenizer, context, question, max_length, text):
        self.context = context
        self.question = question
        self.text = text
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.context)
  
    def __getitem__(self, idx):
        context_ = self.context[idx]
        question_ = self.question[idx]
        text_ = self.text[idx]
    
        #encoding
        input_ids = self.tokenizer.encode(question_, context_)
        answer_ids = self.tokenizer.encode(text_)
        token_type_ids = [0 if i <= input_ids.index(6) else 1 for i in range(len(input_ids))]
    
        #calculating start and end position of the answer in input_ids
        s_pos, e_pos = 0, 0
        for i in range(len(input_ids)):
            if (input_ids[i: i+len(answer_ids[1:-1])] == answer_ids[1:-1]):
                s_pos = i
                e_pos = i + len(answer_ids[1:-1]) - 1
                break

        assert((s_pos<len(input_ids)) & (e_pos<len(input_ids)) & (s_pos<=e_pos))

        if (len(input_ids) < self.max_length):
            padding_len = self.max_length - len(input_ids)
            ids = input_ids + ([0]*padding_len)
        else:
            ids = input_ids[:self.max_length]

        if (len(token_type_ids)<self.max_length):
            padding_len = self.max_length - len(token_type_ids)
            token_ids = token_type_ids  + ([1]*padding_len)
        else:
            token_ids = token_type_ids[:self.max_length]

        return {'ids': torch.tensor(ids, dtype = torch.long),
                'token_type_ids': torch.tensor(token_ids, dtype = torch.long),
                'start_pos': torch.tensor(s_pos, dtype = torch.long),
                'end_pos': torch.tensor(e_pos, dtype = torch.long)}

# Training and evaluation function


In [0]:
def train_loop(dataloader, model, optimizer, device, max_grad_norm, scheduler=None):
    model.train()
    for bi, d in enumerate(tqdm_notebook(dataloader, desc="Iteration")):
        ids = d['ids']
        # mask_ids = d['mask']
        token_ids = d['token_type_ids']
        start_pos = d['start_pos']
        end_pos = d['end_pos']

        ids = ids.to(device, dtype = torch.long)
        # mask_ids = mask_ids.to(device, dtype = torch.long)
        token_ids = token_ids.to(device, dtype = torch.long)
        start_pos = start_pos.to(device, dtype = torch.long)
        end_pos = end_pos.to(device, dtype = torch.long)

        optimizer.zero_grad()
        start_and_end_scores = model(ids)
        # start_scores, end_scores = model(ids, token_ids)
        loss = loss_func(start_and_end_scores, start_pos, end_pos)
        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        loss.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()
        if bi%100==0:
            print (f"bi: {bi}, loss: {loss}")

In [0]:
def eval_loop(dataloader, model, device):
    model.eval()
    pred_s = None
    pred_e = None
    eval_loss = 0.0
    eval_steps = 0

    for bi, d in enumerate(dataloader):
        ids = d['ids']
        # mask_ids = d['mask']
        token_ids = d['token_type_ids']
        start_pos = d['start_pos']
        end_pos = d['end_pos']

        ids = ids.to(device, dtype = torch.long)
        # mask_ids = mask_ids.to(device, dtype = torch.long)
        token_ids = token_ids.to(device, dtype = torch.long)
        start_pos = start_pos.to(device, dtype = torch.long)
        end_pos = end_pos.to(device, dtype = torch.long)

        with torch.no_grad():
            start_and_end_scores = model(ids)
#             start_and_end_scores = model(ids, token_ids)

            loss = loss_func(start_and_end_scores, start_pos, end_pos)
            eval_loss += loss.mean().item()

        eval_steps += 1
        if pred_s is None:
            pred_s = start_and_end_scores[0].detach().cpu().numpy()
            pred_e = start_and_end_scores[1].detach().cpu().numpy()
        else:
            pred_s = np.append(pred_s, start_and_end_scores[0].detach().cpu().numpy(), axis=0)
            pred_e = np.append(pred_e, start_and_end_scores[1].detach().cpu().numpy(), axis=0)

    eval_loss = eval_loss/eval_steps
    pred_start = np.argmax(pred_s, axis=1)
    pred_end = np.argmax(pred_e, axis=1)

    return eval_loss, pred_start, pred_end

# Configuration

In [0]:
MAX_SEQ_LENGTH = 512
TRAIN_BATCH_SIZE = 16  # 16, 32  it was 8
EVAL_BATCH_SIZE = 16 # 8 WAS
LEARNING_RATE = 1e-5 #1e-5 or 5e-5
NUM_TRAIN_EPOCHS = 3 # or 2, 3, -> 4
BERT_TYPE = "camembert-base"#"fmikaelian/camembert-base-fquad"
max_grad_norm = 1.0

In [0]:
tokenizer = CamembertTokenizer.from_pretrained(BERT_TYPE, do_lower_case = True)

train_dataset = BertDatasetModule(
    tokenizer = tokenizer,
    context = train_data['context'],
    question = train_data['question'],
    max_length = MAX_SEQ_LENGTH,
    text = train_data['text']
)

train_dataloader = DataLoader(train_dataset, batch_size = TRAIN_BATCH_SIZE, shuffle=True)

In [0]:
eval_dataset = BertDatasetModule(
    tokenizer = tokenizer,
    context = valid_data['context'],
    question = valid_data['question'],
    max_length = MAX_SEQ_LENGTH,
    text = valid_data['text']
)

eval_dataloader = DataLoader(eval_dataset, batch_size = EVAL_BATCH_SIZE, shuffle=False)

In [58]:
# import transformers
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model = CamemBERTQA(bert_type = BERT_TYPE, hidden_size = 768, num_labels = 2, n_blocks = 2, n_heads = 12).to(device)
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, correct_bias=False)

NUM_TRAIN_STEPS = int(len(train_dataset)/TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS) 
scheduler = transformers.get_constant_schedule_with_warmup(
                optimizer,
                num_warmup_steps=500,
#                 num_training_steps=NUM_TRAIN_STEPS,
                last_epoch=-1)

cuda


In [29]:
NUM_TRAIN_STEPS

1225

In [30]:
model.eval()


CamemBERTQA(
  (camembert): CamembertModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-0

In [0]:
# # Get all of the model's parameters as a list of tuples.
# params = list(model.named_parameters())

# print('The BERT model has {:} different named parameters.\n'.format(len(params)))

# print('==== Embedding Layer ====\n')

# for p in params[0:5]:
#     print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

# print('\n==== First Transformer ====\n')

# for p in params[5:21]:
#     print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

# print('\n==== Encoder Layer ====\n')

# for p in params[21:]:
#     print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

# Training Iterations


In [0]:
#training
for epoch in trange(NUM_TRAIN_EPOCHS):
    train_loop(train_dataloader, model, optimizer, device, max_grad_norm, scheduler)

  0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


bi: 0, loss: 12.445550918579102
bi: 100, loss: 9.353384017944336
bi: 200, loss: 8.478275299072266
bi: 300, loss: 6.773229122161865


 33%|███▎      | 1/3 [03:01<06:02, 181.39s/it]

bi: 0, loss: 10.100625038146973
bi: 100, loss: 10.137435913085938
bi: 200, loss: 8.533740997314453
bi: 300, loss: 8.445728302001953


 67%|██████▋   | 2/3 [06:02<03:01, 181.33s/it]

bi: 0, loss: 6.583135604858398
bi: 100, loss: 7.353538990020752


# Evaluation

In [56]:
res = eval_loop(eval_dataloader, model, device)
print(res[0])

Token indices sequence length is longer than the specified maximum sequence length for this model (10 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (13 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (13 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (10 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (15 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this mod

12.427672094569768


## Test


In [46]:
# path_ali_2 = r"SQuAD-v1.1-dev.json"
path_ali_2= r"valid.json"
dev_data = formatting_squad(path_ali_2)
# train_data = train_data[train_data['context'].apply(lambda x: len(x)<500)].reset_index(drop=True)
print(dev_data.shape)
dev_data.head(3)

(3188, 4)


,context,question,answer_start,text
0,Les deux tableaux sont certes décrits par des ...,Que concerne principalement les documents ?,161,La Vierge aux rochers
1,Les deux tableaux sont certes décrits par des ...,Par quoi sont décrit les deux tableaux ?,46,documents contemporains
2,Les deux tableaux sont certes décrits par des ...,Quels types d'objets sont les deux tableaux au...,204,objets de spéculations


In [47]:


context_ = dev_data['context']
question_ = dev_data['question']
text_ = dev_data['text']
pred_start = res[1]
pred_end = res[2]
res_text_ = []
act_start = []
act_end = []


input_ids_list = list(map(lambda x,y: tokenizer.encode(x, y), question_, context_))
answer_ids_list = list(map(lambda x: tokenizer.encode(x), text_))

for i in range(len(input_ids_list)):
    res_text_.append(tokenizer.decode(input_ids_list[i][pred_start[i]:pred_end[i]+1]))
    s_pos, e_pos = 0, 0
    for j in range(len(input_ids_list[i])):
        if (input_ids_list[i][j: j+len(answer_ids_list[i][1:-1])] == answer_ids_list[i][1:-1]):
            s_pos = j
            e_pos = j + len(answer_ids_list[i][1:-1]) - 1
            break
    act_start.append(s_pos)
    act_end.append(e_pos)

Token indices sequence length is longer than the specified maximum sequence length for this model (10 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (13 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (13 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (10 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (15 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this mod

In [0]:
dev_data['start_pos'] = act_start
dev_data['end_pos'] = act_end
dev_data['predicted_text'] = res_text_
dev_data['predicted_start_pos'] = pred_start
dev_data['predicted_end_pos'] = pred_end

In [49]:
show_columns = ['text', 'predicted_text', 'start_pos', 'end_pos', 'predicted_start_pos', 'predicted_end_pos']
dev_data[show_columns].head(20)

,text,predicted_text,start_pos,end_pos,predicted_start_pos,predicted_end_pos
0,La Vierge aux rochers,la vierge aux rochers,39,42,39,42
1,documents contemporains,documents contemporains à leur création,20,21,20,24
2,objets de spéculations,la vierge aux rochers,56,59,47,50
3,droite,jambe droite,62,62,61,62
4,gauche,leur pied gauche,63,63,61,63
5,vert,vert,176,176,176,176
6,atelier de Léonard de Vinci,l'atelier de léonard de vinci,0,0,26,35
7,séchage,processus de séchage,126,126,124,126
8,doigts,doigts,91,91,91,91
9,La Vierge aux rochers,<s> quel est le nom du panneau central du reta...,0,0,0,67


In [50]:
cond1 = dev_data['predicted_start_pos']>dev_data['predicted_end_pos']
cond2 = dev_data['end_pos']<dev_data['predicted_start_pos']
cond3 = dev_data['start_pos']>dev_data['predicted_end_pos']

incorrect_pred = dev_data[(cond1) | (cond2) | (cond3)].shape[0]
incorrect_pred

910

In [51]:
t = dev_data.shape[0]
print(f"accuracy = {(t - incorrect_pred)*100/t}")

accuracy = 71.45545796737767


In [0]:
dev_data[(cond1) | (cond2) | (cond3)][show_columns].head()


,text,predicted_text,start_pos,end_pos,predicted_start_pos,predicted_end_pos
2,objets de spéculations,la vierge aux rochers,56,59,47,50
6,atelier de Léonard de Vinci,l'atelier de léonard de vinci,0,0,26,35
9,La Vierge aux rochers,(la vierge aux rochers),0,0,63,68
40,adhérence,problèmes d'adhérence,0,0,33,36
44,la tête,,74,75,74,45


In [0]:
csv_incorrect = dev_data[(cond1) | (cond2)| (cond3)][show_columns]
csv_correct = dev_data.drop(csv_incorrect.index)[show_columns]

In [0]:
csv_incorrect.to_csv('csv_incorrect.csv')
csv_correct.to_csv('csv_correct.csv')